In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [10]:
# Define the Deep Neural Network (DNN) for the PIML model
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
#Defining the PINN model
class PhysicsInformedModel(nn.Module):
    def __init__(self, dnn, w, R):
        super(PhysicsInformedModel, self).__init__()
        self.dnn = dnn
        self.w = w  # Angular velocity
        self.R = R  # Radius of the system

    def forward(self, inputs):
        return self.dnn(inputs)

    def physics_loss(self, phi, vs, mu, gamma,p):
        # Physics loss based on the given equations for M and Q
        #torque
        M = self.w * self.R**2 * torch.trapz(mu * vs*p, phi)
        #energies
        Q_a = self.w * self.R * torch.trapz(mu * vs*p, phi)
        Q_b = self.w * self.R * torch.trapz(gamma * vs**2, phi)
        
        # Total dissipation as the physics constraint
        physics_loss = torch.abs(M - (Q_a + Q_b))
        return physics_loss

In [12]:
# Hyperparameters and dummy data setup
input_dim = 3  # (phi, vs, mu)
hidden_dim = 10
output_dim = 1  # Torque M prediction
epochs = 1000
learning_rate = 0.001
w = 1.0  # Dummy value for angular velocity
R = 1.0  # Dummy value for radius

In [13]:
# Initialize the DNN and PIML model
dnn = DNN(input_dim, hidden_dim, output_dim)
model = PhysicsInformedModel(dnn, w, R)

In [14]:
# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Generate random dummy data with requires_grad=True
num_samples = 100
phi = torch.linspace(0, 2 * np.pi, num_samples).reshape(-1, 1).requires_grad_(True)
vs = torch.rand(num_samples, 1, requires_grad=True)  # Random slip speed values
mu = torch.rand(num_samples, 1, requires_grad=True)  # Random friction coefficient values
gamma = torch.rand(num_samples, 1, requires_grad=True)  # Random internal friction values
p = torch.rand(num_samples, 1, requires_grad=True)  # Random pressure values

In [16]:
# Compute physics-informed loss
physics_loss = model.physics_loss(phi, vs, mu, gamma,p)

# Take the mean (or sum) of the loss to get a scalar
physics_loss = physics_loss.mean()

# Backward pass and optimization
physics_loss.backward()
optimizer.step()

In [17]:
print("Physics Loss Shape:", physics_loss.shape)

Physics Loss Shape: torch.Size([])


In [18]:
# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass through DNN
    inputs = torch.cat([phi, vs, mu], dim=1)
    outputs = model(inputs)
    
    # Calculate the physics-informed loss
    physics_loss = model.physics_loss(phi, vs, mu, gamma,p)
    
    # Ensure the physics_loss is a scalar
    physics_loss = physics_loss.mean() 

    # Backward pass and optimization
    physics_loss.backward()
    optimizer.step()
    
    # Logging
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Physics Loss: {physics_loss.item()}")

print("Training complete.")

Epoch 0, Physics Loss: 0.0
Epoch 100, Physics Loss: 0.0
Epoch 200, Physics Loss: 0.0
Epoch 300, Physics Loss: 0.0
Epoch 400, Physics Loss: 0.0
Epoch 500, Physics Loss: 0.0
Epoch 600, Physics Loss: 0.0
Epoch 700, Physics Loss: 0.0
Epoch 800, Physics Loss: 0.0
Epoch 900, Physics Loss: 0.0
Training complete.
